# __CatBoost__
É um algoritmo baseado em árvores de decisão, que usa a técnica de Gradient Boosting para trabalhar com problemas de classificação, regressão e ranking. É um algoritmo muito utilizado principalmente pela capacidade de lidar com variáveis __categóricas__, fazendo com que não seja necessário aplicar alguns pré-processamentos como OneHotEncoding.

## __Conceito e Intuição__
Ele é baseado no Gradient Boosting, mas apresenta alguns pontos importantes:
- 1) __Capacidade de utilizar dados categóricos:__ Ele é um algoritmo que converte as variáveis categóricas automaticamente em representação numéricas através de um método baseado em contagem (_statistics-based encoding_).
- 2) __Redução do overfitting:__ Aplica técnicas como o _Ordered Boosting_ para evitar o vazamento de dados de treino e validação durante a construção das árvores.
- 3) __Eficiência computacional:__ Foi desenvolvido para usar menos memória quando comparadao a algoritmos como XGBoost e LightGBM.

A principal intuição por trás do CatBoost é combinar as vantagens do Gradient Boost com métodos de pré-processamento, mesmo quando lidando com muitas variáveis categóricas.

## __Funcionamento__
#### __Gradient Boosting__
Como mencionado antes, o CatBoost é baseado no Gradient Boosting, onde um conjunto de vários modelos fracos (árvores de decisão) é ajustado sequencialmente, onde cada modelo seguinte tenta corrigir o erro do modelo anterior.

#### __Ordered Boosting__
Uma das grandes inovações aqui é o uso do Ordered Boosting, que lida com o problema de vazamento de dados. Em algoritmos como o XGBoost, as árvores são construídas com acesso a todo o conjunto de dados de treino, gerando a possibilidade de overfitting.

Já no __CatBoost__, cada modelo é treinado usando apenas dados que estariam disponíveis em tempo real. Para cada exemplo, as estatísticas das variáveis categóricas são calculadas sem incluir informações fututras, criando um particionamento dos dados.

Isso é obtido através da divisão do conjunto de treino em folds de forma sequencial.

#### __Encoding de Variáveis Categóricas__
É realizada a transformação das variáveis categóricas usando _contagens estatísticas condicionalmente dependentes de outras variáveis_:
- A média da variável target é calculada para cada categoria, aplicando regularização para evitar overfitting.
- Isso faz com que não seja necessário aplicar técnicas como OHE, reduzindo a dimensionalidade do conjunto.

## __Pré-processamento__
- 1) Como o CatBoost já lida com variáveis categóricas, é necessário apenas que a gente especifique quais são as variáveis categóricas.
- 2) Ele também lida automaticamente com valores nulos, fazendo com que não seja necessária imputação prévia.
- 3) Não precisamos realizar normalização ou padronização das variáveis pois o CatBoost não é sensível a escala dos dados.

In [1]:
# Importação do google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importações
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [4]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.4 MB/s eta 0:00:00


In [5]:
from catboost import CatBoostClassifier, Pool

In [6]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [7]:
alvo = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas em numéricas manualmente, sem escalonamento
previsores = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart2.pkl')

# previsores_esc = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart3.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas em numéricas pelo LabelEncoder.
previsores2 = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart4.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas em numéricas pelo LabelEncoder e OneHotEncoder, sem escalonamento.
previsores3 = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart5.pkl')

# Variáveis previsoras onde as variáveis categóricas foram transformadas pelo LabelEncoder e OHE, com escalonamento.
previsores3_esc = pd.read_pickle('/content/drive/MyDrive/Udemy/ML com Python/1 - Aprendizado Supervisionado: Classificacao/heart6.pkl')

In [8]:
X_tr, X_ts, y_tr, y_ts = train_test_split(previsores, alvo, test_size=.3, random_state=42)

In [12]:
# Criação do Pool de dados para otimização do CatBoost
train_pool = Pool(X_tr, y_tr)
test_pool = Pool(X_ts, y_ts)

In [13]:
# Instanciamento do modelo
catboost = CatBoostClassifier(iterations=100, learning_rate=0.1,
                              depth=6, random_state=42, verbose=10)

# Ajuste do modelo aos dados
catboost.fit(train_pool, eval_set=test_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6494612	test: 0.6507178	best: 0.6507178 (0)	total: 46.8ms	remaining: 4.63s
10:	learn: 0.4393579	test: 0.4586415	best: 0.4586415 (10)	total: 54ms	remaining: 437ms
20:	learn: 0.3663464	test: 0.3943694	best: 0.3943694 (20)	total: 60.5ms	remaining: 228ms
30:	learn: 0.3248692	test: 0.3609986	best: 0.3609986 (30)	total: 67.6ms	remaining: 150ms
40:	learn: 0.3009616	test: 0.3420592	best: 0.3420592 (40)	total: 74.3ms	remaining: 107ms
50:	learn: 0.2825338	test: 0.3305141	best: 0.3305141 (50)	total: 82.3ms	remaining: 79.1ms
60:	learn: 0.2655466	test: 0.3196211	best: 0.3196211 (60)	total: 89.3ms	remaining: 57.1ms
70:	learn: 0.2518731	test: 0.3133904	best: 0.3133904 (70)	total: 96.6ms	remaining: 39.5ms
80:	learn: 0.2394161	test: 0.3102986	best: 0.3099123 (78)	total: 104ms	remaining: 24.4ms
90:	learn: 0.2325303	test: 0.3111033	best: 0.3099123 (78)	total: 111ms	remaining: 11ms
99:	learn: 0.2264399	test: 0.3102598	best: 0.3099123 (78)	total: 118ms	remaining: 0us

bestTest = 0.3099123466
be

In [14]:
# Realização das previsões
y_pred = catboost.predict(X_ts)

In [15]:
# Exibição da acurácia
acc = accuracy_score(y_ts, y_pred)
print(f'Acurácia: {acc}')

Acurácia: 0.8804347826086957


In [16]:
# Exibição da matriz de confusão
print(confusion_matrix(y_ts, y_pred))

[[ 94  13]
 [ 20 149]]


In [17]:
# Exibição do report
print(classification_report(y_ts, y_pred))

              precision    recall  f1-score   support

           0       0.82      0.88      0.85       107
           1       0.92      0.88      0.90       169

    accuracy                           0.88       276
   macro avg       0.87      0.88      0.88       276
weighted avg       0.88      0.88      0.88       276



## Aplicação de Validação cruzada e teste de diferentes parâmetros

In [20]:
strat_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [21]:
resultado = cross_val_score(catboost, previsores, alvo, cv=strat_kfold)
print(resultado.mean())

0:	learn: 0.6488694	total: 724us	remaining: 71.7ms
10:	learn: 0.4352350	total: 8.54ms	remaining: 69.1ms
20:	learn: 0.3613905	total: 16.1ms	remaining: 60.5ms
30:	learn: 0.3240380	total: 23.4ms	remaining: 52.1ms
40:	learn: 0.2962614	total: 32.8ms	remaining: 47.2ms
50:	learn: 0.2763500	total: 40.7ms	remaining: 39.1ms
60:	learn: 0.2639928	total: 49.1ms	remaining: 31.4ms
70:	learn: 0.2570483	total: 57.3ms	remaining: 23.4ms
80:	learn: 0.2478349	total: 64.9ms	remaining: 15.2ms
90:	learn: 0.2430491	total: 72.6ms	remaining: 7.18ms
99:	learn: 0.2393802	total: 79.1ms	remaining: 0us
0:	learn: 0.6493218	total: 606us	remaining: 60.1ms
10:	learn: 0.4411786	total: 8.56ms	remaining: 69.3ms
20:	learn: 0.3611885	total: 16.4ms	remaining: 61.6ms
30:	learn: 0.3186935	total: 24.2ms	remaining: 53.8ms
40:	learn: 0.2926519	total: 33ms	remaining: 47.5ms
50:	learn: 0.2731181	total: 40.1ms	remaining: 38.6ms
60:	learn: 0.2601359	total: 47.4ms	remaining: 30.3ms
70:	learn: 0.2506861	total: 55.1ms	remaining: 22.5ms
80

In [23]:
# Testar diferentes parâmetros
params = {
    'iterations': np.arange(50, 501, 50),
    'learning_rate': np.arange(0.01, 0.11, 0.01),
    'depth': np.array([4, 6, 8, 10, 11]),
    'l2_leaf_reg': np.arange(1, 11, 1),
    'bagging_temperature': np.array([0, 0.5, 1]),
    'subsample': np.array([.5, .7, 1]),
    'random_strength': np.array([0, 2, 3, 5, 10])
}

In [24]:
random_search = RandomizedSearchCV(
    estimator=catboost,
    param_distributions=params,
    n_iter=50,
    cv=strat_kfold,
    random_state=42
)

In [25]:
random_search.fit(previsores, alvo)

Streaming output truncated to the last 5000 lines.
200:	learn: 0.0391660	total: 446ms	remaining: 442ms
210:	learn: 0.0370224	total: 468ms	remaining: 419ms
220:	learn: 0.0349120	total: 493ms	remaining: 399ms
230:	learn: 0.0334231	total: 520ms	remaining: 381ms
240:	learn: 0.0320193	total: 543ms	remaining: 358ms
250:	learn: 0.0297429	total: 566ms	remaining: 336ms
260:	learn: 0.0273240	total: 590ms	remaining: 314ms
270:	learn: 0.0249949	total: 616ms	remaining: 293ms
280:	learn: 0.0233613	total: 650ms	remaining: 275ms
290:	learn: 0.0221195	total: 674ms	remaining: 252ms
300:	learn: 0.0211631	total: 700ms	remaining: 230ms
310:	learn: 0.0206292	total: 722ms	remaining: 207ms
320:	learn: 0.0200063	total: 742ms	remaining: 183ms
330:	learn: 0.0197067	total: 758ms	remaining: 158ms
340:	learn: 0.0189989	total: 780ms	remaining: 135ms
350:	learn: 0.0184630	total: 803ms	remaining: 112ms
360:	learn: 0.0179864	total: 828ms	remaining: 89.4ms
370:	learn: 0.0176283	total: 859ms	remaining: 67.2ms
380:	learn:

RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
                   estimator=<catboost.core.CatBoostClassifier object at 0x794089367fd0>,
                   n_iter=50,
                   param_distributions={'bagging_temperature': array([0. , 0.5, 1. ]),
                                        'depth': array([ 4,  6,  8, 10, 11]),
                                        'iterations': array([ 50, 100, 150, 200, 250, 300, 350, 400, 450, 500]),
                                        'l2_leaf_reg': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]),
                                        'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ]),
                                        'random_strength': array([ 0,  2,  3,  5, 10]),
                                        'subsample': array([0.5, 0.7, 1. ])},
                   random_state=42)

In [26]:
print(f"Melhores parâmetros: {random_search.best_params_}")
print(f"Melhor desempenho: {random_search.best_score_}")

Melhores parâmetros: {'subsample': 0.7, 'random_strength': 0, 'learning_rate': 0.05, 'l2_leaf_reg': 6, 'iterations': 200, 'depth': 4, 'bagging_temperature': 0.0}
Melhor desempenho: 0.8887443573295319


In [27]:
# Aplicação dos melhores parâmetros aos dados de treino sem validação cruzada
best_catboost = CatBoostClassifier(
    iterations=random_search.best_params_['iterations'],
    learning_rate=random_search.best_params_['learning_rate'],
    depth=random_search.best_params_['depth'],
    l2_leaf_reg=random_search.best_params_['l2_leaf_reg'],
    bagging_temperature=random_search.best_params_['bagging_temperature'],
    subsample=random_search.best_params_['subsample'],
    random_strength=random_search.best_params_['random_strength'],
    random_state=42,
    verbose=10
)

In [28]:
best_catboost.fit(X_tr, y_tr)

0:	learn: 0.6495831	total: 863us	remaining: 172ms
10:	learn: 0.4308927	total: 8.1ms	remaining: 139ms
20:	learn: 0.3594221	total: 15.1ms	remaining: 128ms
30:	learn: 0.3224675	total: 22.2ms	remaining: 121ms
40:	learn: 0.2992417	total: 29.2ms	remaining: 113ms
50:	learn: 0.2817078	total: 36ms	remaining: 105ms
60:	learn: 0.2735732	total: 42.8ms	remaining: 97.4ms
70:	learn: 0.2622171	total: 49.5ms	remaining: 90ms
80:	learn: 0.2524101	total: 56.6ms	remaining: 83.2ms
90:	learn: 0.2443235	total: 65ms	remaining: 77.9ms
100:	learn: 0.2372088	total: 72.4ms	remaining: 70.9ms
110:	learn: 0.2311301	total: 78.9ms	remaining: 63.3ms
120:	learn: 0.2268092	total: 86.5ms	remaining: 56.5ms
130:	learn: 0.2191043	total: 93.6ms	remaining: 49.3ms
140:	learn: 0.2115939	total: 101ms	remaining: 42.1ms
150:	learn: 0.2085297	total: 107ms	remaining: 34.8ms
160:	learn: 0.2041293	total: 115ms	remaining: 27.9ms
170:	learn: 0.1991269	total: 122ms	remaining: 20.7ms
180:	learn: 0.1944233	total: 129ms	remaining: 13.5ms
190:

In [29]:
y_pred = best_catboost.predict(X_ts)
print(f'Acurácia: {accuracy_score(y_ts, y_pred)}')

Acurácia: 0.8659420289855072


In [30]:
# Matriz de confusão
print(confusion_matrix(y_ts, y_pred))

[[ 92  15]
 [ 22 147]]


In [31]:
# Report de classificação
print(classification_report(y_ts, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.86      0.83       107
           1       0.91      0.87      0.89       169

    accuracy                           0.87       276
   macro avg       0.86      0.86      0.86       276
weighted avg       0.87      0.87      0.87       276

